# IMPORT IMPORTANT LIBS

In [ ]:
%pip install requirements.txt

In [164]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# IMPORT DATASET

In [187]:
# Train set
url_train = "../data/raw/train.csv"
df_train = pd.read_csv(url_train)

# Test set
url_test = "../data/raw/test.csv"
df_test_set = pd.read_csv(url_test)

## Data Cleaning

### Checking Data Attributes

In [166]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [188]:
df_test_set.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [189]:
# Data Set Shape
print("Train set shape:", df_train.shape)
print("Test set shape:", df_test_set.shape)

Train set shape: (1460, 81)
Test set shape: (1459, 80)


### Finding Columns with Missing Values

In [172]:
def get_missing_columns(df, df_name="DataFrame"):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    
    if not missing.empty:
        print(f"\nMissing values in {df_name}:")
        print(missing)
    else:
        print(f"\nNo missing values in {df_name}.")
    
    return list(missing.index)


In [190]:
missing_train = get_missing_columns(df_train, df_name="df_train")
missing_test = get_missing_columns(df_test_set, df_name="df_test")


Missing values in df_train:
LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

Missing values in df_test:
MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType       894
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
Garage

In [191]:
print("Missing Test Values", len(missing_test))
print("Missing Train Values", len(missing_train))

Missing Test Values 33
Missing Train Values 19


### Train Set

In [193]:
# Assume df is your original dataframe
# Split data first (before cleaning)
df_train, df_test = train_test_split(df_train, test_size=0.2, random_state=42)

In [194]:
# Function to clean dataset
def clean_data(df):
    df = df.copy()

    # Fill LotFrontage with mean
    df["LotFrontage"].fillna(df["LotFrontage"].mean(), inplace=True)

    # Drop columns with too many missing values
    df.drop(columns=["Alley", "MasVnrType", "PoolQC", "Fence", "MiscFeature"], inplace=True, errors='ignore')

    # Fill MasVnrArea with mean
    df["MasVnrArea"].fillna(df["MasVnrArea"].mean(), inplace=True)

    # Fill basement-related columns with "NA"
    for col in ["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2"]:
        df[col].fillna("NA", inplace=True)

    # Fill Electrical with "NA"
    df["Electrical"].fillna("NA", inplace=True)

    # Fill garage-related missing values
    df["GarageYrBlt"].fillna(df["GarageYrBlt"].median(), inplace=True)
    for col in ["GarageType", "GarageFinish", "GarageQual", "GarageCond"]:
        df[col].fillna("NA", inplace=True)

    return df

In [195]:

# Clean both datasets separately
df_train_cleaned = clean_data(df_train)
df_test_cleaned = clean_data(df_test)

C:\Users\HP\AppData\Local\Temp\ipykernel_17868\2645672042.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["LotFrontage"].fillna(df["LotFrontage"].mean(), inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_17868\2645672042.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

In [196]:
print("Train set missing values:")
missing_train = df_train_cleaned.isna().sum()
missing_train = missing_train[missing_train > 0]
print(missing_train)


Train set missing values:
FireplaceQu    429
dtype: int64


In [197]:
# Checking if nana values are present
print("Cleaned Train set shape:", df_train_cleaned.shape)
print("Cleaned Test set shape:", df_test_cleaned.shape)

Cleaned Train set shape: (934, 76)
Cleaned Test set shape: (234, 76)


In [198]:
# Dropping FireplaceQu column as it has too many missing values
# We address later for model performance
df_train_cleaned.drop(columns=["FireplaceQu"], inplace=True, errors='ignore')
df_test_cleaned.drop(columns=["FireplaceQu"], inplace=True, errors='ignore')

In [199]:
#  Checking if nana values are present after cleaning
print(df_train_cleaned.isna().sum().sum())
print(df_test_cleaned.isna().sum().sum())

0
0


### Train Set To CSV

In [200]:
# Save cleaned training data
df_train_cleaned.to_csv("../data/cleaned/df_train_cleaned.csv", index=False)

# Save cleaned test data
df_test_cleaned.to_csv("../data/cleaned/df_test_cleaned.csv", index=False)


## TesSet

In [201]:
def clean_test_data(df):
    # Fill numerical with mean
    df["LotFrontage"].fillna(df["LotFrontage"].mean(), inplace=True)
    df["GarageYrBlt"].fillna(df["GarageYrBlt"].median(), inplace=True)

    # Fill categorical with "NA"
    for col in ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", 
                "BsmtFinType1", "BsmtFinType2", "GarageType", "GarageQual"]:
        df[col].fillna("NA", inplace=True)

    # Drop high-missing or unneeded columns (ignore errors if missing)
    df.drop(columns=["FireplaceQu", "Alley", "MasVnrType", "PoolQC", "Fence", "MiscFeature"], inplace=True, errors='ignore')

    return df


In [202]:
# Clean the test data separately
df_test_set_cleaned = clean_test_data(df_test_set.copy())

C:\Users\HP\AppData\Local\Temp\ipykernel_17868\1117628795.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["LotFrontage"].fillna(df["LotFrontage"].mean(), inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_17868\1117628795.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [205]:
print("Shape", df_test_set_cleaned.shape)
print("Train Set Cleaned Shape", df_train_cleaned.shape)

Shape (1459, 74)
Train Set Cleaned Shape (934, 75)


### Verifying Columns

In [207]:
# Print shapes
print("Shape of df_test_set_cleaned:", df_test_set_cleaned.shape)
print("Shape of df_train_cleaned:", df_train_cleaned.shape)

# Check if columns match
test_cols = set(df_test_set_cleaned.columns)
train_cols = set(df_train_cleaned.columns)

if test_cols == train_cols:
    print("✅ Columns match in name.")
else:
    print("❌ Columns do not match.")

    only_in_train = train_cols - test_cols
    only_in_test = test_cols - train_cols

    if only_in_train:
        print("Columns only in df_train_cleaned:", list(only_in_train))
    if only_in_test:
        print("Columns only in df_test_set_cleaned:", list(only_in_test))


Shape of df_test_set_cleaned: (1459, 74)
Shape of df_train_cleaned: (934, 75)
❌ Columns do not match.
Columns only in df_train_cleaned: ['SalePrice']


### Test Set to csv

In [ ]:
# Test Set Cleaned to CSV
df_test_set_cleaned.to_csv("../data/cleaned/test_cleaned.csv", index=False)

In [227]:
df_test_set_cleaned.loc[df_test_set_cleaned["YrSold"] > 2020, "YrSold"]

Series([], Name: YrSold, dtype: int64)

In [219]:
df_test_set_cleaned["YearBuilt"].dtype

dtype('int64')